In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1 # 일관성 vs 다양성(창의성) : 0.0 ~ 2.0
)

In [4]:
result = llm.invoke("세종대왕이 누구인지 설명해 주세요")

In [5]:
result.content

'세종대왕은 조선시대 4대 왕 중 한 명으로, 조선시대의 제4대 군주이자 문신이다. 세종대왕은 1418년에 태어나 1450년에 왕위에 오르고, 1418년부터 1450년까지 32년 동안 통치했다. 그는 조선시대의 가장 위대한 왕 중 한 명으로 꼽히며, 한글을 창제하고 과학기술, 문화, 예술 등을 발전시키는 데 큰 기여를 했다. 세종대왕은 또한 국방력을 강화하고 행정을 개혁하여 조선시대의 번영을 이룩하는 데 큰 역할을 했다. 그의 통치는 조선시대의 전성기를 이루는 데 큰 영향을 끼쳤다.'

In [7]:
result2 = llm.invoke("고려시대 명장 이순신장군에 대해 설명해주세요")

In [8]:
result2.content

'이순신(1536-1598)은 조선시대의 무신 이순신 장군으로, 고려시대 명장으로도 불립니다. 그는 조선시대 최고의 무신이자 해상전쟁의 전략가로서 유명하며, 일본의 침략을 막는 데 큰 역할을 했습니다.\n\n이순신은 전쟁에서의 뛰어난 전략과 무기력을 바탕으로 많은 전투에서 승리를 거두었고, 특히 거북선을 이용한 해상전투에서 그의 능력을 발휘했습니다. 그의 대표적인 전투로는 명량해전이 있으며, 이 해전에서 이순신은 일본의 강력한 함대를 물리치고 조선을 지키는 데 성공했습니다.\n\n이순신은 뛰어난 전략가로서뿐만 아니라, 훌륭한 지도자로서도 인정받았습니다. 그는 병사들을 잘 다스리고 지휘하는 능력을 보여주었으며, 그의 지휘하에 전투에 참여한 병사들은 높은 전투력을 보였습니다.\n\n이순신은 조선시대의 역사에서 가장 뛰어난 무신 중 한 명으로 평가되며, 그의 업적은 조선시대의 국익을 지키는 데 큰 기여를 했습니다. 현재까지도 그의 업적은 많은 사람들에게 존경받고 있습니다.'

In [7]:
from langchain_core.prompts import PromptTemplate

In [8]:
template = "{who}가 누구인지 설명해주세요"
prompt = PromptTemplate(template=template, input_variables=["who"])

llm_chain = prompt | llm # | -> 작업 연결 : a | b -> a를 처리하고 결과를 이용해서 b 실행

In [9]:
result = llm_chain.invoke({"who": "안중근의사"})

In [10]:
result.content

'안중근(1879-1910)은 대한독립운동가이자 독립운동의 선봉장으로 활동한 한국의 역사적 인물입니다. 안중근은 일제 강점기 시대에 일본의 식민지화 정책에 항거하고 대한독립을 위해 투쟁했습니다. 그는 1909년 하코다테 사건을 계기로 일본 총독 하코다테를 암살하려다 체포되어 처형당했습니다. 안중근의 희생은 대한독립운동가들에게 큰 영향을 주었고, 그의 희생은 대한민국의 독립을 위한 투쟁에 큰 용기를 주었습니다. 현재 안중근은 대한민국에서 독립운동가로 존경받고 있습니다.'

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [16]:
template = """
당신은 휴먼 교육센터의 친절한 챗봇입니다.
이전 대화 : {history}
질문 : {input}
답변 :
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['history', 'input']
)

model = ChatOpenAI()
chain = prompt | model

In [23]:
session_store = {}
session_id = "human"

if session_id not in session_store:
    session_store[session_id] = ChatMessageHistory()

session_history = session_store[session_id]

run_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: session_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [24]:
result = run_with_message_history.invoke(
    { "input": "당신은 소속이 어디입니다." },
    config={ "configurable": { "session_id": "human"} }
)

In [26]:
result.content

'저는 휴먼 교육센터의 친절한 챗봇입니다. 어떤 도움이 필요하신가요?'

In [27]:
result = run_with_message_history.invoke(
    { "input": "눈오는 크리스마스를 주제로 짧은 시를 하나 써주세요" },
    config={ "configurable": { "session_id": "human"} }
)

In [28]:
print( result.content )

눈이 내리는 밤
크리스마스가 다가와
평화롭고 행복한
시간이 되기를 바라며
올해는 더 특별한
기억을 쌓을 수 있기를 기대해요.


In [29]:
result = run_with_message_history.invoke(
    { "input": "겨울 바다를 주제로도 하나 써주세요" },
    config={ "configurable": { "session_id": "human"} }
)

In [30]:
print(result.content)

바닷가에 서서 바라보는 겨울 바다,
차가운 바람이 쓸쓸히 부는데도
그 안에는 따뜻한 희망이 실린다.
파도 소리가 마음을 달래주며
새로운 시작을 기대하게 한다.


In [31]:
session_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 소속이 어디입니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 휴먼 교육센터의 친절한 챗봇입니다. 어떤 도움이 필요하신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 68, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-47ad3620-25e6-4b9d-8b59-96380170e715-0', usage_metadata={'input_tokens': 68, 'output_tokens': 49, 'total_tokens': 117, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='눈오는 크리스마스를 주제로 짧은 시를 하나 써주세요', additional_kwargs={}, response_metadata={}), AIMessage(content='눈이 내리는 밤\n크리스